In [1]:
import geopandas as gpd
import pandas as pd
from os import listdir
import re
from collections import Counter

shapefile = 'maps/ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country', 'country_code3', 'geometry']
gdf.head()

,country,country_code3,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [2]:
print(gdf[gdf['country'] == 'Antarctica'])
#Drop row corresponding to 'Antarctica'
gdf = gdf.drop(gdf.index[159])

        country country_code3  \
159  Antarctica           ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


In [3]:
path = '../luxoft/source/'
files = listdir(path)

df_res = pd.read_csv(path + '/' + files[-1])
df_res.columns = ['Specialization',' Position / Title', 'Location', 'none']

# for k in files[:-1]:
#     df = pd.read_csv(path + '/' + k).drop("num", axis=1)
#     df = df.rename(columns={"res": "res" + k})
#     df_res = pd.merge(df_res, df, how='outer', on='Country Name')

df_res['country_code2'] = df_res['Location'].str.split('/ ', expand=True)[1]
df_res = pd.DataFrame(Counter(df_res.country_code2).items(), columns=['country_code2', 'res'])
df_res.head()

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [ ]:
cc = pd.read_csv('maps/country_codes.csv')
df_res = cc.merge(df_res, how='left', on='country_code2')

In [ ]:
#Merge dataframes gdf and df_res
merged = gdf.merge(df_res, how='left', on='country_code3')
merged.head()

In [ ]:
#Add rank
original_res = merged.res.dropna().sort_values()
merged['rank_res'] = merged['res'].rank()

In [ ]:
import json
#Read data to json.
merged_json = json.loads(merged.to_json())#Convert to String like object.
json_data = json.dumps(merged_json)

In [ ]:
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import Magma256

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Reverse color order so that dark blue is highest obesity.
palette = Magma256[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = len(original_res))
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'luxoft', height = 600 , width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'rank_res', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)